# Проект "Исследование данных об инвестиции венчурных фондов в компании-стартапы"

Проект автоматически проверяется в тренажёре SQL. Работа идёт с базой данных, которая хранит информацию о венчурных фондах и инвестициях в компании-стартапы. В проекте 23 задания на составление запросов к БД.

### Цель проекта

Произвести различные выгрузки данных венчурных фондов с помощью SQL

 ### ER-диаграмма
 
<img src='Image.png' width=700, heigth=500>

### Описание таблиц

`acquisition` - содержит информацию о покупках одних компаний другими

`company` - содержит информацию о компаниях-стартапах

`education` - хранит информацию об уровне образования сотрудников компаний

`fund`- хранит информацию о венчурных фондах

`funding_round` - содержит информацию о раундах инвестиций

`investment`- содержит информацию об инвестициях венчурных фондов в компании-стартапы

`people`- содержит информацию о сотрудниках компаний-стартапов

### Выполнение заданий

#### 1. Отобразите все записи из таблицы `company` по компаниям, которые закрылись
``` sql
SELECT COUNT(id)
FROM company
WHERE status = 'closed';
```

#### 2. Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы `company`. Отсортируйте таблицу по убыванию значений в поле `funding_total`
``` sql
SELECT funding_total
FROM company
WHERE category_code = 'news'
AND country_code = 'USA'
ORDER BY funding_total DESC;
```

#### 3. Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно
``` sql
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash'
AND EXTRACT(YEAR FROM CAST(acquired_at AS date)) BETWEEN 2011 AND 2013;
```

#### 4. Отобразите имя, фамилию и названия аккаунтов людей в поле network_username, у которых названия аккаунтов начинаются на `Silver`
``` sql
SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%';
```

#### 5. Выведите на экран всю информацию о людях, у которых названия аккаунтов в поле` network_username` содержат подстроку 'money', а фамилия начинается на 'K'
``` sql
SELECT *
FROM people
WHERE twitter_username LIKE '%money%'
AND last_name LIKE 'K%';
```

#### 6. Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы
``` sql
SELECT country_code,
       SUM(funding_total)
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC;
```

#### 7. Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату. Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению
``` sql
SELECT funded_at,
       MIN(raised_amount),
       MAX(raised_amount)
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount)!=0
AND MIN(raised_amount)!= MAX(raised_amount);
```

#### 8. Создайте поле с категориями:
* Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию `high_activity`
* Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию `middle_activity`
* Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию `low_activity`

**Отобразите все поля таблицы `fund` и новое поле с категориями**
``` sql
SELECT *,
    CASE
    WHEN invested_companies >=100 THEN 'high_activity'
    WHEN invested_companies <20 THEN 'low_activity'
    ELSE 'middle_activity'
    END
FROM fund;
```

#### 9. Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего
``` sql
SELECT CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
       ROUND(AVG(investment_rounds)) AS avg_round
FROM fund
GROUP BY activity
ORDER BY avg_round;
```

#### 10. Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 
* Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. 
* Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке
``` sql
SELECT country_code,
       MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies)
FROM fund
WHERE founded_at BETWEEN '2010-01-01' AND '2012-12-31'
GROUP BY country_code
HAVING MIN(invested_companies)!=0
ORDER BY AVG(invested_companies) DESC, country_code
LIMIT 10;
```

#### 11. Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна
``` sql
SELECT p.first_name,
       p.last_name,
       e.instituition
FROM people AS p
LEFT JOIN education AS e ON p.id=e.person_id;
```

#### 12. Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов
``` sql
SELECT c.name AS company_name,
       COUNT(DISTINCT e.instituition) AS instituition_count
FROM company AS c
JOIN people AS p ON c.id=p.company_id
JOIN education AS e ON p.id=e.person_id
GROUP BY company_name
ORDER BY instituition_count DESC
LIMIT 5;
```

#### 13. Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним
``` sql
SELECT DISTINCT name 
FROM company AS c
JOIN (SELECT company_id
            FROM funding_round
            WHERE is_first_round=1
            AND is_last_round=1) AS fr
            ON c.id=fr.company_id
WHERE status = 'closed'; 
```

#### 14. Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании
``` sql
WITH
total AS (SELECT DISTINCT name,
      c.id
FROM company AS c
JOIN (SELECT company_id
      FROM funding_round
      WHERE is_first_round=1
      AND is_last_round=1) AS fr ON c.id=fr.company_id
WHERE status = 'closed')
SELECT p.id
FROM people AS p
JOIN total ON total.id=p.company_id;
```

#### 15. Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник
``` sql
WITH
total AS (SELECT DISTINCT name,
      c.id
FROM company AS c
JOIN (SELECT company_id
      FROM funding_round
      WHERE is_first_round=1
      AND is_last_round=1) AS fr ON c.id=fr.company_id
WHERE status = 'closed')
SELECT DISTINCT p.id,
       e.instituition
FROM people AS p
JOIN total ON total.id=p.company_id
JOIN education AS e ON p.id=e.person_id;
```

#### 16. Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды
``` sql
WITH
total AS (SELECT DISTINCT name,
      c.id
FROM company AS c
JOIN (SELECT company_id
      FROM funding_round
      WHERE is_first_round=1
      AND is_last_round=1) AS fr ON c.id=fr.company_id
WHERE status = 'closed')
SELECT DISTINCT p.id,
       COUNT(e.instituition)
FROM people AS p
JOIN total ON total.id=p.company_id
JOIN education AS e ON p.id=e.person_id
GROUP BY p.id;
```

#### 17. Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится
``` sql
SELECT AVG(count)
FROM (WITH
total AS (SELECT DISTINCT name,
      c.id
FROM company AS c
JOIN (SELECT company_id
      FROM funding_round
      WHERE is_first_round=1
      AND is_last_round=1) AS fr ON c.id=fr.company_id
WHERE status = 'closed')
SELECT DISTINCT p.id,
       COUNT(e.instituition)
FROM people AS p
JOIN total ON total.id=p.company_id
JOIN education AS e ON p.id=e.person_id
GROUP BY p.id) AS baza;
```

#### 18. Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Socialnet
``` sql
SELECT AVG(count)
FROM (WITH
total AS (SELECT c.id
      FROM company AS c
      WHERE name = 'Socialnet')
SELECT  p.id,
        COUNT(e.instituition)
FROM people AS p
JOIN total ON total.id=p.company_id
JOIN education AS e ON p.id=e.person_id
GROUP BY p.id) AS baza;
```

#### 19. Составьте таблицу из полей:
* `name_of_fund` — название фонда
* `name_of_company` — название компании
* `amount` — сумма инвестиций, которую привлекла компания в раунде

**В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно**
``` sql
SELECT name_of_fund,
       name_of_company,
       fr.raised_amount AS amount
FROM (SELECT funding_round_id,
             f.name AS name_of_fund,
             c.name AS name_of_company
FROM investment AS i
INNER JOIN fund AS f ON f.id = i.fund_id
INNER JOIN company AS c ON c.id=i.company_id
WHERE c.milestones>6) AS baza
INNER JOIN funding_round AS fr ON fr.id=baza.funding_round_id
WHERE EXTRACT(YEAR FROM CAST(funded_at AS date)) BETWEEN 2012 AND 2013;
```

#### 20. Выгрузите таблицу, в которой будут такие поля:
* название компании-покупателя;
* сумма сделки;
* название компании, которую купили;
* сумма инвестиций, вложенных в купленную компанию;
* доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

**Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы.**

**Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.**
``` sql
SELECT c.name,
       a.price_amount,
       cc.name,
       cc.funding_total,
       ROUND(a.price_amount/cc.funding_total)
FROM acquisition AS a
LEFT JOIN company AS c ON a.acquiring_company_id=c.id
LEFT JOIN company AS cc ON a.acquired_company_id=cc.id
WHERE a.price_amount>0
AND cc.funding_total>0
ORDER BY a.price_amount DESC, cc.name
LIMIT 10;
```

#### 21. Выгрузите таблицу, в которую войдут названия компаний из категории `social`, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования
``` sql
SELECT c.name,
       EXTRACT (MONTH FROM funded_at) AS month
FROM company AS c
LEFT JOIN funding_round AS fr ON c.id=fr.company_id
WHERE c.category_code='social'
AND EXTRACT (YEAR FROM funded_at) BETWEEN 2010 AND 2013
AND fr.raised_amount != 0;
```

#### 22. Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
* номер месяца, в котором проходили раунды;
* количество уникальных названий фондов из США, которые инвестировали в этом месяце;
* количество компаний, купленных за этот месяц;
* общая сумма сделок по покупкам в этом месяце.
``` sql
WITH
f AS (SELECT EXTRACT (MONTH FROM CAST(funded_at AS date)) AS month,
             COUNT(DISTINCT fund_id) AS fund_count
      FROM investment AS i
      JOIN funding_round AS fr ON i.funding_round_id=fr.id
      WHERE EXTRACT (YEAR FROM CAST(funded_at AS date)) BETWEEN '2010' AND '2013' 
      AND fund_id IN (SELECT id
                      FROM fund
                      WHERE country_code = 'USA')
      GROUP BY month),

a AS (SELECT EXTRACT(MONTH FROM CAST(acquired_at AS date)) AS month_acquired,
             COUNT(acquired_company_id) AS acquired_count,
             SUM(price_amount) AS sum_price
      FROM acquisition
      WHERE EXTRACT(YEAR FROM CAST(acquired_at AS date)) BETWEEN '2010' AND '2013'
      GROUP BY month_acquired)

SELECT month,
       fund_count,
       acquired_count,
       sum_price
FROM f
JOIN a ON f.month=a.month_acquired;
```

#### 23. Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему
``` sql
WITH
inv_2011 AS (SELECT country_code,
                 AVG(funding_total) AS avg_2011
            FROM company
            WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2011
            GROUP BY country_code),  
inv_2012 AS (SELECT country_code,
                 AVG(funding_total) AS avg_2012
            FROM company
            WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2012
            GROUP BY country_code),
inv_2013 AS (SELECT country_code,
                 AVG(funding_total) AS avg_2013
            FROM company
            WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2013
            GROUP BY country_code)
SELECT inv_2011.country_code,
       avg_2011,
       avg_2012,
       avg_2013
FROM inv_2011
INNER JOIN inv_2012 ON inv_2011.country_code=inv_2012.country_code
INNER JOIN inv_2013 ON inv_2011.country_code=inv_2013.country_code
ORDER BY avg_2011 DESC;
```